## Compile Projects
To-do
* Figure out how to version things b/c projects will get updated and we want to track any changes.
* This only needs to be done with data from lp2000 and ctips.
* Need to track changes across all the different dataframes
* Use merges to figure it out?

In [1]:
import _harmonization_utils as har_utils
import _lrtp_utils as lrtp_utils
import _sb1_utils as sb1_utils
import _specific_list_utils
import _state_rail_plan_utils as srp_utils
import geopandas as gpd
import pandas as pd
from calitp_data_analysis.sql import to_snakecase
import hashlib
from datetime import datetime

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_list/"

### LRTP/LOST

In [19]:
lrtp_lost_df, lrtp_lost_gdf = lrtp_utils.all_mpo(True)

/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract

96 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:720: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sandag.cost2020m.str.replace("$", "")
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


65 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


360 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


68 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract

In [21]:
def unique_project_number(df:pd.DataFrame) -> pd.DataFrame:
    df['timestamp'] = datetime.now().strftime('%Y%m%d%H%M%S')
    df['combo'] = df.apply(lambda row: f"{row.name}{row['timestamp']}{''.join(map(str, row))}", axis=1)
    df['project_number'] = df['combo'].apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest()[:12])
    df = df.drop(columns = ['combo', 'timestamp'])
    return df

In [22]:
lrtp_lost_df = unique_project_number(lrtp_lost_df)

In [23]:
lrtp_lost_df.project_number.nunique()

16276

In [55]:
def separate_out_df(df:pd.DataFrame, columns_to_keep: list)-> pd.DataFrame:
    """
    Subset the column, drop any rows 
    in which the values are Nan or "None."
    """
    # Subset
    df2 = df[columns_to_keep]
    
    # Fill in missing values
    try:
        df2 = df2.fillna('none')
    except:
        df2
    
    # Remove project_number and keep only cols of interest
    columns_to_keep.remove('project_number')
    
    # Drop rows that are nan or "None" based on how many columns are listed
    if len(columns_to_keep) == 1:
        df2 = df2.dropna(how = "any")
        df2 = df2[df2.applymap(lambda x: x.lower() if isinstance(x, str) else x) != 'none'].dropna()
        
    # If there are more than one column to separate out,
    # keep any row that has a non-null value 
    else:
        df2 = df2.dropna(how = "all", subset = columns_to_keep)
    return df2

In [25]:
lrtp_lost_county = separate_out_df(lrtp_lost_df, ['project_number', 'county'])

In [26]:
lrtp_lost_df.shape

(16276, 11)

In [27]:
lrtp_lost_county.shape

(4012, 2)

In [28]:
lrtp_lost_city = separate_out_df(lrtp_lost_df, ['project_number', 'city'])

In [29]:
lrtp_lost_city.shape

(745, 2)

In [30]:
lrtp_lost_agency = separate_out_df(lrtp_lost_df, ['project_number', 'lead_agency'])

In [31]:
lrtp_lost_agency.shape

(16276, 2)

In [32]:
lrtp_lost_agency.sample()

,project_number,lead_agency
61,8d1631fce5bc,Slocog


In [33]:
lrtp_lost_geo = separate_out_df(lrtp_lost_df, ['project_number', 'geometry'])

In [34]:
lrtp_lost_geo.shape, lrtp_lost_gdf.shape

((1357, 2), (1355, 10))

In [35]:
lrtp_to_drop = ['county', 'city', 'lead_agency', 'geometry']

In [36]:
# lrtp_lost_df = lrtp_lost_df.drop(columns = lrtp_to_drop)

In [37]:
lrtp_lost_df.sample()

,project_title,lead_agency,project_year,project_description,total_project_cost,geometry,city,county,data_source,notes,project_number
416,None,Stancog,2022-2046,"Bicycle Lane (Class 2), Buffered Bicycle Lane (Class 2), Bicycle Route With Wide Shoulders (Class 3.5), Separated Bike Lane (Class 4), And Pedestrian Improvements.\n(Non-Motorized Transportation Plan Top 25: Route 25)",8027400.00,None,None,None,Stancog Lrtp,"Jurisdiction: Stanislaus County, Location: Rhode Rd/7Th St/Nunes Rd/N. Golden State Blvd, Project Limits: Moore Rd To W. Christofferson Pkwy, Funding Source: Atp, Sb 1, Bil/Iija, Cmaq, Stbgp, System Preserv : Nan, Capacity Enhance : Nan, Safety: Nan, Oper : Nan, Complete Streets: Nan, Active\nTransporta Tion: X, Transit: Nan, Other: Nan",a32f75c83b70


### SB1

In [38]:
sb1_df = sb1_utils.load_sb1()

/home/jovyan/data-analyses/project_list/_sb1_utils.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  gdf[i]
/home/jovyan/data-analyses/project_list/_sb1_utils.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  gdf[i]


0 rows are mising geometry
7917 rows contain invalid geography


In [40]:
sb1_df = unique_project_number(sb1_df)

In [76]:
sb1_df.sample()

,projectid,projname,projcatcode,projcategory,projprogcode,projprogram,multiprogfunded,projstatus,description,cost,assemblydistrict,senatedistrict,assemblycode,senatecode,countyname,cityname,countycode,citycode,appagencyname,impagencyname,geometry,totalcosts,routes,constyear,costfull,projagency,project_number,assembly_same,senate_same
3449,LsrFy17185261Pp030,Slurry Seal Parker Avenue,Local,Local And Regional,Lsr1718,201718 Local Streets And Roads,NaN,PreConstruction,Slurry Seal 16324 Sf For Roadway With 69 Pci,NaN,24,13,24,13,San Mateo,Atherton Submitted By City,Sm,Atn,NaN,NaN,None,NaN,NaN,2018,NaN,Atherton City,c61c443fb21d,True,True


In [41]:
len(sb1_df)

9186

In [42]:
sb1_df.project_number.nunique()

9186

#### Check that assemblydistrict and assemblycode are the same values

In [43]:
sb1_df['assembly_same'] = sb1_df.assemblycode == sb1_df.assemblydistrict

In [44]:
sb1_df.assembly_same.value_counts()

True     7217
False    1969
Name: assembly_same, dtype: int64

In [45]:
sb1_df['senate_same'] = sb1_df.senatedistrict == sb1_df.senatecode

In [46]:
sb1_df.senate_same.value_counts()

True     6950
False    2236
Name: senate_same, dtype: int64

In [47]:
sb1_county = separate_out_df(sb1_df, ['project_number', 'countyname'])

In [48]:
def explode_dataframe(df:pd.DataFrame, column_to_explode:str)-> pd.DataFrame:
    df['Column2'] = df[column_to_explode].apply(lambda x: [int(i) if i.isdigit() else i for i in x.replace(',', '').split()])
    df = df.drop(columns = [column_to_explode])
    df2 = df.explode('Column2')
    df2 = df2.rename(columns = {'Column2': column_to_explode})
    return df2

In [49]:
sb1_assembly = separate_out_df(sb1_df, ['project_number', 'assemblydistrict'])

In [50]:
sb1_assembly = explode_dataframe(sb1_assembly, 'assemblydistrict')

In [51]:
sb1_senate = separate_out_df(sb1_df, ['project_number', 'senatedistrict'])

In [52]:
sb1_senate = explode_dataframe(sb1_senate, 'senatedistrict')

In [53]:
sb1_city = separate_out_df(sb1_df, ['project_number', 'cityname'])

In [56]:
sb1_geo = separate_out_df(sb1_df, ['project_number', 'geometry'])

In [57]:
sb1_awards = separate_out_df(sb1_df, ['project_number', 'projprogram'])

In [58]:
sb1_awards.shape, sb1_geo.shape, sb1_city.shape

((9186, 2), (1585, 2), (6696, 2))

In [59]:
sb1_agencies = separate_out_df(sb1_df, ['project_number', 'projagency', 'appagencyname', 'impagencyname'])

In [60]:
sb1_agencies.shape

(9186, 4)

### LP2000

In [79]:
def load_lp2000(file: str):
    
    df_project = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="project")
    )

    df_county = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="county")
    ).drop(columns=["project_label_name"])

    df_district = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="district")
    ).drop(columns=["project_label_name"])

    df_award = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="awards")
    )
    
    df_phase = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="phase_funding")
    )
    
    return df_project, df_county, df_district, df_award, df_phase

In [80]:
lp2000_project, lp2000_county, lp2000_district, lp2000_award, lp2000_phase = load_lp2000("LP2000_projects.xlsx")

In [63]:
# lp2000_project = unique_project_number(lp2000_project)

In [64]:
len(lp2000_project), lp2000_project.project_number.nunique()

(11272, 11272)

In [75]:
def add_project_number(df_with_project_name:pd.DataFrame, right_project:pd.DataFrame, merge_col:str)-> pd.DataFrame:
    m1 = pd.merge(df_with_project_name, right_project, on = merge_col, how = 'inner')
    return m1

In [71]:
lp2000_project_subset = lp2000_project[['project_number', 'project_id']]

In [72]:
lp2000_county_df = pd.merge(lp2000_project_subset, lp2000_county, on = 'project_id', how = 'inner')

In [73]:
lp2000_county_df.sample()

,project_number,project_id,county_name
4377,0b952b66e020,5202(007),Los Angeles County


In [74]:
lp2000_project.loc[lp2000_project.project_id == "5202(007)"]

,project_id,comment_desc,est_total_prj_costs,location_name,project_label_name,original_post_mile_begin_id,original_post_mile_end_id,revised_post_mile_begin_ind,revised_post_mile_end_ind,route_name,state_hwy_ind,senate_district_code,update_date_time,agency_name,urban_area_code,work_type_desc,category_desc,current_phase,active_transportation_program__atp_,bridge_inspection___scour_evaluation,covid_relief_funds_for_highway_infrastructure_programs_for_stip_covid_augmentation,carbon_reduction_program__crp_,congestion_mitigation___air_quality_improvement_program__cmaq_,coronavirus_response_and_relief_supplemental_appropriations_act__crrsaa__funds,corridor_mobility_improvement_account__cmia__program,county_exchange_funds,county_state_match_program,"earmarks_projects__hpp,_demo_cpfcds,_etc__",emergency_relief__er_,ferry_boat_program__fbp__and_ferry_boat_discretionary__fbd__program,"funds_for_planning,_programming_and_monitoring___rip",general_funded_designated_programs,hazard_elimination_safety__hes_,high_risk_rural_roads_program__hr3_,highway_bridge_,highway_safety_improvement_program__hsip___infrastructure__state_fund,highway_safety_improvement_program__hsip___non_infrastructure_,highway_safety_improvement_program__hsip__infrastructure__federal_fund,local_partnership_program__lpp_–_competitive__,local_roads,local_roads_rehabilitation,railroad_grade_crossing_protection,railroad_grade_separations,"rebuilding_american_infrastructure_with_sustainability_and_equity__raise__and_multimodal_project_discretionary_grant_programs__e_g_,_infra,_mega,_rstg_or_rural__",regional_improvement_program_–_regional_share_of_stip_transportation_enhancement__off_system_,regional_surface_transportation_block_grant_program__rstbgp__and_highway_infrastructure_program__hip_,regional_transportation_planning_agency__rtpa__stp_match_exchange,sb1_funded_freeway_service_patrol,shopp__traffic_light_synchronization_program__tlsp___proposition_1b_bond_funds,safe_routes_to_school__sr2s_and_srts_,set_aside_coordinated_border_infrastructure__cbi__program_under_fast_act,solutions_for_congested_corridors_program__sccp_,special_programs,state_local_partnership_program__slpp__and_local_partnership_program__lpp_formulaic_,structures_seismic_retrofit_,trade_corridor_enhancement_account__tcea__programs_–_local_share,trade_corridor_enhancement_account__tcea__programs_–_state_share,trade_corridors_improvement_fund__tcif__program_local_streets___roads,traffic_congestion_relief_program___tcrp__,project_number
4377,5202(007),file has been transferred to Saad issa on 12/14/05. Mike Benyamin\n\nInactive- ms,NaN,Various Locations Citywide,Traffic Sign Upgrades,NaN,NaN,NaN,NaN,0-SFR,N,NaN,2010-08-09 20:04:27,San Fernando,3041.00,NaN,Signs,single phase,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,0b952b66e020


In [77]:
len(lp2000_project), lp2000_project.project_id.nunique()

(11272, 11272)

In [81]:
len(lp2000_project.drop(columns = ['project_id']).drop_duplicates())

11263

In [82]:
11272-11263

9

### CTIPS

In [ ]:
def load_ctips(file: str):
  
    df_project = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="project")
    )

    df_county = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="county")
    )

    df_district = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="district")
    )

    df_phase = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="phase_funding")
    )
    
    df_award = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="awards")
    )
    
    df_house = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="us_house")
    )
    
    df_senate = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="senate")
    )
    
    df_assembly = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="assembly")
    )
    return df_project, df_county, df_district, df_phase, df_award, df_house, df_senate, df_assembly

In [ ]:
ctips_project, ctips_county, ctips_district, ctips_phase, ctips_award, ctips_house, ctips_senate, ctips_assembly = load_ctips('CTIPS.xlsx')

In [ ]:
ctips_project.sample()

### State Rail Plan

In [ ]:
srp_df = har_utils.load_state_rail_plan()

In [ ]:
srp_df = generate_alphanumeric_ids(srp_df, 8)

In [ ]:
srp_df.sample()

In [ ]:
srp_df_agency = separate_out_df(srp_df, ['project_number', 'lead_agency'])